In [68]:
import pandas as pd
import numpy as np
from pydataset import data
from sqlalchemy import text, create_engine

from env import get_connection

In [189]:
url  = get_connection('employees')
engine = create_engine(url)


In [3]:
query = '''
            SELECT *  

            FROM employees'''

In [4]:
text_query = text(query)

In [5]:
df = pd.read_sql(text_query, engine.connect())
df.head()

,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12


In [10]:
df_employees = df
df_employees


,emp_no,birth_date,first_name,last_name,gender,hire_date
0,10001,1953-09-02,Georgi,Facello,M,1986-06-26
1,10002,1964-06-02,Bezalel,Simmel,F,1985-11-21
2,10003,1959-12-03,Parto,Bamford,M,1986-08-28
3,10004,1954-05-01,Chirstian,Koblick,M,1986-12-01
4,10005,1955-01-21,Kyoichi,Maliniak,M,1989-09-12
...,...,...,...,...,...,...
300019,499995,1958-09-24,Dekang,Lichtner,F,1993-01-12
300020,499996,1953-03-07,Zito,Baaz,M,1990-09-27
300021,499997,1961-08-03,Berhard,Lenart,M,1986-04-21
300022,499998,1956-09-05,Patricia,Breugel,M,1993-10-13


5. Once you have successfully run a query:

    a. Intentionally make a typo in the database url. What kind of error message do you see?
    
        OperationalError: (pymysql.err.OperationalError) (2003, "Can't connect to MySQL server on 'data.codeup.dcom' ([Errno 8] nodename nor servname provided, or not known)")

    
    b. Intentionally make an error in your SQL query. What does the error message look like?

        [SQL: 
            SELECT *  

            FROM emplodyees]


6. Read the employees and titles tables into two separate DataFrames.


In [6]:
query = '''
            SELECT *  

            FROM titles'''

In [7]:
text_query = text(query)
df_titles = pd.read_sql(text_query, engine.connect())
df_titles.head()

,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01


7. How many rows and columns do you have in each DataFrame? Is that what you expected?

         300024 for employees, and 443308 for titles. Yes because one employee can have multiple     titles thorugh out their employment


In [11]:
df_employees.shape

(300024, 6)

In [9]:
df_titles.shape

(443308, 4)

8. Display the summary statistics for each DataFrame.


In [15]:
df_employees.describe()
df_titles.describe()

,emp_no
count,443308.000000
mean,253075.034430
std,161853.292613
min,10001.000000
25%,84855.750000
50%,249847.500000
75%,424891.250000
max,499999.000000


9. How many unique titles are in the titles DataFrame?


In [28]:
df_titles['title'].nunique()

7

10. What is the oldest date in the to_date column? 


In [39]:
df_titles['to_date'].min().strftime('%Y-%m-%d')

'1985-03-01'

11. What is the most recent date in the to_date column?

In [38]:
df_titles['to_date'].max().strftime('%Y-%m-%d')

'9999-01-01'

## Exercises II

1. Copy the users and roles DataFrames from the examples above.

In [40]:
# Create the roles DataFrame

roles = pd.DataFrame({
    'id': [1, 2, 3, 4],
    'name': ['admin', 'author', 'reviewer', 'commenter']
})
roles


,id,name
0,1,admin
1,2,author
2,3,reviewer
3,4,commenter


In [45]:
# Create the users DataFrame.

users = pd.DataFrame({
    'id': [1, 2, 3, 4, 5, 6],
    'name': ['bob', 'joe', 'sally', 'adam', 'jane', 'mike'],
    'role_id': [1, 2, 3, 3, np.nan, np.nan]
})
users


,id,name,role_id
0,1,bob,1.0
1,2,joe,2.0
2,3,sally,3.0
3,4,adam,3.0
4,5,jane,NaN
5,6,mike,NaN


2. What is the result of using a right join on the DataFrames?

        drop users with no roll id, keep roles even wieh no users


In [48]:
users.merge(roles, how='right', left_on= 'role_id', right_on= 'id')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1,admin
1,2.0,joe,2.0,2,author
2,3.0,sally,3.0,3,reviewer
3,4.0,adam,3.0,3,reviewer
4,NaN,NaN,NaN,4,commenter


3. What is the result of using an outer join on the DataFrames?

          keeps all data, users with no roles AND roles with no users


In [49]:
users.merge(roles, how='outer', left_on= 'role_id', right_on= 'id')

,id_x,name_x,role_id,id_y,name_y
0,1.0,bob,1.0,1.0,admin
1,2.0,joe,2.0,2.0,author
2,3.0,sally,3.0,3.0,reviewer
3,4.0,adam,3.0,3.0,reviewer
4,5.0,jane,NaN,NaN,NaN
5,6.0,mike,NaN,NaN,NaN
6,NaN,NaN,NaN,4.0,commenter


4. What happens if you drop the foreign keys from the DataFrames and try to merge them?

        KeyError: 'id'


In [65]:
roles = roles.drop(columns='id')

In [66]:
roles

,name
0,admin
1,author
2,reviewer
3,commenter


In [67]:
users.merge(roles, how='inner', left_on= 'role_id', right_on= 'id')

KeyError: 'id'

5. Load the mpg dataset from PyDataset. 


In [73]:
mpg = data('mpg')
mpg

,manufacturer,model,displ,year,cyl,trans,drv,cty,hwy,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


6. Output and read the documentation for the mpg dataset.


In [70]:
data('mpg', show_doc=True)

mpg

PyDataset Documentation (adopted from R Documentation. The displayed examples are in R)

## Fuel economy data from 1999 and 2008 for 38 popular models of car

### Description

This dataset contains a subset of the fuel economy data that the EPA makes
available on http://fueleconomy.gov. It contains only models which had a new
release every year between 1999 and 2008 - this was used as a proxy for the
popularity of the car.

### Usage

    data(mpg)

### Format

A data frame with 234 rows and 11 variables

### Details

  * manufacturer. 

  * model. 

  * displ. engine displacement, in litres 

  * year. 

  * cyl. number of cylinders 

  * trans. type of transmission 

  * drv. f = front-wheel drive, r = rear wheel drive, 4 = 4wd 

  * cty. city miles per gallon 

  * hwy. highway miles per gallon 

  * fl. 

  * class. 




7. How many rows and columns are in the dataset?


In [76]:
mpg.shape()

TypeError: 'tuple' object is not callable

8. Check out your column names and perform any cleanup you may want on them.


In [84]:
mpg = mpg.rename(columns= {'cty' : 'city', 'hwy':'highway', 'trans':'transmission', 'cyl':'cylinders'})
mpg

,manufacturer,model,displ,year,cylinders,transmission,drv,city,highway,fl,class
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact
...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize


9. Display the summary statistics for the dataset.


In [86]:
mpg.describe()

,displ,year,cylinders,city,highway
count,234.000000,234.000000,234.000000,234.000000,234.000000
mean,3.471795,2003.500000,5.888889,16.858974,23.440171
std,1.291959,4.509646,1.611534,4.255946,5.954643
min,1.600000,1999.000000,4.000000,9.000000,12.000000
25%,2.400000,1999.000000,4.000000,14.000000,18.000000
50%,3.300000,2003.500000,6.000000,17.000000,24.000000
75%,4.600000,2008.000000,8.000000,19.000000,27.000000
max,7.000000,2008.000000,8.000000,35.000000,44.000000


10. How many different manufacturers are there?


In [133]:
mpg.manufacturer.nunique()

15

11. How many different models are there?


In [136]:
mpg.model.nunique()

38

12. Create a column named mileage_difference like you did in the DataFrames exercises; this column should contain the difference between highway and city mileage for each car.

In [143]:
mpg['mileage_difference'] = mpg.highway - mpg.city
mpg

,manufacturer,model,displ,year,cylinders,transmission,drv,city,highway,fl,class,mileage_difference
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10
...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8


13. Create a column named average_mileage like you did in the DataFrames exercises; this is the mean of the city and highway mileage.


In [172]:
mpg['average_mileage'] = (round((mpg.city + mpg.highway)/2,2))
mpg

,manufacturer,model,displ,year,cylinders,transmission,drv,city,highway,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,23.5,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,25.0,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,25.5,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,25.5,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,21.0,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,23.5,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,25.0,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,21.0,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,22.0,False


14. Create a new column on the mpg dataset named is_automatic that holds boolean values denoting whether the car has an automatic transmission.


In [156]:
mpg.groupby('transmission').manufacturer.count()

transmission
auto(av)       5
auto(l3)       2
auto(l4)      83
auto(l5)      39
auto(l6)       6
auto(s4)       3
auto(s5)       3
auto(s6)      16
manual(m5)    58
manual(m6)    19
Name: manufacturer, dtype: int64

In [162]:
mpg['is_automatic'] = mpg.transmission.str.startswith('auto')
mpg

,manufacturer,model,displ,year,cylinders,transmission,drv,city,highway,fl,class,mileage_difference,average_mileage,is_automatic
1,audi,a4,1.8,1999,4,auto(l5),f,18,29,p,compact,11,40.3,True
2,audi,a4,1.8,1999,4,manual(m5),f,21,29,p,compact,8,40.3,False
3,audi,a4,2.0,2008,4,manual(m6),f,20,31,p,compact,11,40.3,False
4,audi,a4,2.0,2008,4,auto(av),f,21,30,p,compact,9,40.3,True
5,audi,a4,2.8,1999,6,auto(l5),f,16,26,p,compact,10,40.3,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
230,volkswagen,passat,2.0,2008,4,auto(s6),f,19,28,p,midsize,9,40.3,True
231,volkswagen,passat,2.0,2008,4,manual(m6),f,21,29,p,midsize,8,40.3,False
232,volkswagen,passat,2.8,1999,6,auto(l5),f,16,26,p,midsize,10,40.3,True
233,volkswagen,passat,2.8,1999,6,manual(m5),f,18,26,p,midsize,8,40.3,False


15. Using the mpg dataset, find out which which manufacturer has the best miles per gallon on average?


In [183]:
mpg.groupby(by='manufacturer').average_mileage.max().tail(1)

manufacturer
volkswagen    39.5
Name: average_mileage, dtype: float64

16. Do automatic or manual cars have better miles per gallon?

In [188]:
mpg.groupby(by='is_automatic').average_mileage.mean()


is_automatic
False    22.227273
True     19.130573
Name: average_mileage, dtype: float64

## Excercises III


1. Use your get_db_url function to help you explore the data from the chipotle database. 


In [190]:
url  = get_connection('chipotle')
engine = create_engine(url)


In [204]:
query = '''
            SELECT *  

            FROM orders'''

query = '''
            SHOW DATABASES'''

In [205]:
text_query = text(query)

In [206]:
df_chipotle = pd.read_sql(text_query, engine.connect())
df_chipotle.head()

,id,order_id,quantity,item_name,choice_description,item_price
0,1,1,1,Chips and Fresh Tomato Salsa,nan,$2.39
1,2,1,1,Izze,[Clementine],$3.39
2,3,1,1,Nantucket Nectar,[Apple],$3.39
3,4,1,1,Chips and Tomatillo-Green Chili Salsa,nan,$2.39
4,5,2,2,Chicken Bowl,"[Tomatillo-Red Chili Salsa (Hot), [Black Beans, Rice, Cheese, Sour Cream]]",$16.98


2. What is the total price for each order?


In [234]:
df_chipotle['item_price'] = pd.to_numeric(df_chipotle.item_price.str.replace('$',''))

/var/folders/6c/7ht5b3716k32pz2lmh_pzp8m0000gp/T/ipykernel_7414/4033544467.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  df_chipotle['item_price'] = pd.to_numeric(df_chipotle.item_price.str.replace('$',''))


In [265]:
df_chipotle.groupby('order_id').sum()

/var/folders/6c/7ht5b3716k32pz2lmh_pzp8m0000gp/T/ipykernel_7414/1930759899.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  df_chipotle.groupby('order_id').sum()


,id,quantity,item_price
order_id,,,
1,10,4,11.56
2,5,2,16.98
3,13,2,12.67
4,17,2,21.00
5,21,2,13.70
...,...,...,...
1830,9223,2,23.00
1831,13842,3,12.90
1832,9233,2,13.20


3. What are the most popular 3 items?


In [267]:
df_chipotle['item_name'].value_counts().head(3)

Chicken Bowl           726
Chicken Burrito        553
Chips and Guacamole    479
Name: item_name, dtype: int64

In [252]:
df_chipotle['item_name'].mode()

0    Chicken Bowl
Name: item_name, dtype: object

In [254]:
df_chipotle['item_name'].value_counts().head(1)

Chicken Bowl    726
Name: item_name, dtype: int64

4. Which item has produced the most revenue?


In [269]:
df_chipotle.groupby('item_name').item_price.sum().sort_values(ascending= False).head()

item_name
Chicken Bowl           7342.73
Chicken Burrito        5575.82
Steak Burrito          3851.43
Steak Bowl             2260.19
Chips and Guacamole    2201.04
Name: item_price, dtype: float64

In [273]:
df_chipotle['rev_by_item'].groupby('item_name').sum()

KeyError: 'rev_by_item'

5. Join the employees and titles DataFrames together.


,emp_no,title,from_date,to_date
0,10001,Senior Engineer,1986-06-26,9999-01-01
1,10002,Staff,1996-08-03,9999-01-01
2,10003,Senior Engineer,1995-12-03,9999-01-01
3,10004,Engineer,1986-12-01,1995-12-01
4,10004,Senior Engineer,1995-12-01,9999-01-01
...,...,...,...,...
443303,499997,Engineer,1987-08-30,1992-08-29
443304,499997,Senior Engineer,1992-08-29,9999-01-01
443305,499998,Senior Staff,1998-12-27,9999-01-01
443306,499998,Staff,1993-12-27,1998-12-27


6. For each title, find the hire date of the employee that was hired most recently with that title.


7. Write the code necessary to create a cross tabulation of the number of titles by department. (Hint: this will involve a combination of SQL code to pull the necessary data and python/pandas code to perform the manipulations.)

